In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import redditutils


from string import punctuation
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

%matplotlib inline
%load_ext autoreload
%autoreload 2

Suppress Pandas' automatic conversion of utc column to scientific notation:

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
df = pd.read_csv('showerthoughts.csv')

In [4]:
len(df[df.score > 1]) / len(df)

0.392702

39% of submissions get more than one upvote. This makes "greater than one upvote" a good candidate for our criteria for a good post.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
author          1000000 non-null object
title           1000000 non-null object
subreddit       1000000 non-null object
score           1000000 non-null int64
num_comments    1000000 non-null int64
retrieved_on    1000000 non-null int64
id              1000000 non-null object
created_utc     1000000 non-null int64
selftext        591622 non-null object
dtypes: int64(4), object(5)
memory usage: 68.7+ MB


In [6]:
df['created_utc'] = pd.to_datetime(df['created_utc'], unit='s')

In [7]:
pd.set_option('display.max_colwidth', 999)

In [8]:
df_short = df.iloc[:1000, :]

## Clean the data

In [21]:
df['title'] = df['title'].apply(lambda x: strip_punc(x))

In [22]:
df['title'].apply(lambda x: )

1000000

In [15]:
english_words = set(words.words())

Get rid of any misspellings and non-english words.

In [19]:
def strip_punc(s):
    return ''.join(c for c in s if c not in punctuation)

In [4]:
df['title'] = df['title'].apply(lambda x: redditutils.strip_nums(x))

In [5]:
df.head()

,author,title,subreddit,score,num_comments,retrieved_on,id,created_utc,selftext
0,os_coxae,"Everyone that's ever said ""I'm speechless"" is ...",Showerthoughts,0,0,1466445837,4ixl2n,1463006328,NaN
1,Calrizle,I wonder how many people I've talked to who ha...,Showerthoughts,0,0,1466494743,4lfjxn,1464437036,NaN
2,[deleted],"""Palindrome"" is a let down as a word. Rhinopla...",Showerthoughts,0,0,1466493929,4ldt85,1464398463,[deleted]
3,[deleted],"When you tell someone to keep a secret, it is ...",Showerthoughts,0,0,1466423373,4hmcbg,1462264235,[deleted]
4,[deleted],"Someone had the the time to create the words, ...",Showerthoughts,0,0,1466474842,4k9ngz,1463768968,[deleted]


## Word count vectorization

In [6]:
cv = CountVectorizer(stop_words = 'english', min_df = 2, max_df = .95, strip_accents = 'unicode',
                     encoding = 'ISO-8859-1')

In [7]:
X = cv.fit_transform(df['title'])

In [16]:
cv_df = pd.DataFrame(X.toarray(), columns = cv.get_feature_names())

In [17]:
cv_df.head()

,1닷,1닷com,1닷콤,1점,1점coм,1쫌컴,4ᆞcom,TM,__,___,...,毕业证代办,毕业证办理,永久存档,留信网认证,留学回国人员证明,績單,荜业證,落户购买免税车,诚招代理,靠谱
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
cv_df.sum(axis=1).order(ascending=False).head()